# Load Libraries

In [ ]:
options(rgl.useNULL=TRUE)
library(rgl)
library(usethis)
library(lattice)
library(adespatial)
library(vegan)
library(devtools)
library(phyloseq)
library(microbiomeSeq)
library(DECIPHER)
library(ape)
library(GenomicRanges)
library(SummarizedExperiment)
library(MatrixGenerics)
library(matrixStats)
library(DESeq2)
library(ggplot2)
library(plotly)
library(philr)
library(tidyverse)
library(adespatial)
library(qiime2R)
library(MicrobeR)
library(microbiome)
library(ShortRead)
library(microbiomeSeq)
library(pander)
library(ranacapa)
library(grid)
library(gridExtra)
library(knitr)
library(png)
library(heplots)
library(ggdendro)
library(ggpubr)
library(RColorBrewer)
library(microbiomeutilities)
library(viridis)
library(magrittr)
library(dplyr)
library(ggstatsplot)
library(dada2)
library(phangorn)
library(randomcoloR)
library(Polychrome)
library(microbiome)
library(remotes)
library(upstartr)
library(hrbrthemes)
library(psadd)
library(Seurat)
library(microbiomer)
library(VennDiagram)
library(ggvenn)
library(ggprism)
library(multiplyr)
library(rstatix)
library(janitor)
library(microbiome)
library(microViz)
library(Rtsne)
library(ggh4x)
library(RColorBrewer)
library(reshape2)
library(zipangu)
library(grDevices)
library(ragg)
library(ALDEx2)
library(seqinr)
library(seqateurs)
library(metagMisc)
library(data.table)
library(ggpicrust2)
library(Maaslin2)
library(KEGGREST)
library(PERFect)
library(packrat)
library(NCmisc)
library(plyr)
library(ggVennDiagram)

# Definitions, R Image & Functions

### Definitions

In [ ]:
path <- "/Users/dzakaria/research/GiantTrotoisesMicrobiome/research/data/16s/tartarughe_2022/data_processing/" 
R_image <- paste(path,"20231021-AldabraGiantTortoises-dada2.RData", sep="")
EnvironmentGroup5_sorted <- c("Curieuse\n(Nursery)", "Curieuse\n(Perley)", "Curieuse", "Botanical\nGarden", "Parco\nNatura")
AgeGroup13_sorted <- c("1", "2", "4", "5", "15", "30", "40", "50", "60", "70", "80", "90", "100")
AgeGroup_sorted <- c("Young\n(0-30)", "Mid Age\n(30-70)", "Old\n(70-100)", "Centenarian\n(>100)")
AgeGroup_vennsorted <- c("Mid Age\n(30-70)", "Old\n(70-100)", "Centenarian\n(>100)", "Young\n(0-30)")
AgeGroup_vennsorted_wild <- c("Centenarian\n(>100)","Mid Age\n(30-70)","Old\n(70-100)")
AgeGroup_vennsorted_controlled <- c("Old\n(70-100)","Centenarian\n(>100)","Young\n(0-30)", "Mid Age\n(30-70)")
EnvironmentGroup5_vennsorted <- c("Parco\nNatura", "Curieuse","Curieuse\n(Perley)","Botanical\nGarden", "Curieuse\n(Nursery)")

## Preparing color palettes
mycols_environment <- c("olivedrab", "olivedrab3", "orange4")
mycols_environment2 <- c("orange4", "orange3", "olivedrab")
mycols_environment4 <- c("orange3", "orange4", "olivedrab", "olivedrab2")
mycols_environment5 <- c("orange2","orange3", "orange4", "olivedrab", "olivedrab2")
mycols_agegroup13 <- colorRampPalette(c("wheat3","salmon2"))(13)

### Load R Image

In [ ]:
# The directory containing fastq files after unzipping
setwd(path)
load(file = R_image)

### Functions

In [ ]:
alpha_diversity_calculation <- 
  function(ps){
  ps_df <- phyloseq::psmelt(ps)
  ps_rich <- estimate_richness(ps)
  row.names(ps_rich) <- gsub("X", "", row.names(ps_rich))
  ps_rich$Sample <- row.names(ps_rich)
  ps_df_rich <- merge(ps_rich, ps_df, by = "Sample")
  ps_df_rich$AgeGroup13 <- as.factor(ps_df_rich$AgeGroup13)
  return(ps_df_rich)
}

alpha_diversity_plot <- 
  function(ps_df_rich, xname, yname, ref_group_wilcox, jitter_colors_variable_name, jitter_colors, x_axis_lab, y_axis_lab, plot_title, legend_title){
  xdata <- ps_df_rich[,xname]
  ydata <- ps_df_rich[,yname]
  jitter_colors_variabledata <- ps_df_rich[,jitter_colors_variable_name]
  ggplot(ps_df_rich, aes(x = xdata, y = ydata)) +
    geom_boxplot()+
    geom_jitter(position = position_dodge(0.8), cex = 3, aes(color=jitter_colors_variabledata)) +
    labs(x = x_axis_lab, y= y_axis_lab, title = stringr::str_wrap(plot_title, width = 50)) +
    theme_prism() +
    scale_color_manual(legend_title, values = jitter_colors) +
    theme(
      plot.margin = margin(1, 1, 1, 1, "cm"),
      legend.position = "right",
      axis.text.x = element_text(color = "black", size = 25),
      axis.text.y = element_text(color = "black", size = 25),
      axis.title = element_text(size = 30),
      legend.title = element_text(size = 30),
      legend.text = element_text(size = 25),
      strip.text.x = element_text(size = 28),
      plot.title = element_text(size = 40)) +
    stat_compare_means(
      aes(label = after_stat(p.signif)),
      method = "wilcox.test",
      ref.group = ref_group_wilcox)
  }

ps_rel_abund_taxacore_createdf <- function(ps_rel_abun, taxa_name){
  taxa_name_capitalized <- str_to_title(taxa_name)
  if(taxa_name_capitalized == "Phylum" | taxa_name_capitalized == "Species"){
    taxa_variables <- c(
      "taxa_name_capitalized" = taxa_name_capitalized,
      "prevalence" =as.integer(0))
  } else {
    taxa_variables <- c(
      "taxa_name_capitalized" = taxa_name_capitalized,
      "prevalence" =as.integer(0.5))
  }
  ps_rel_abund_taxacore <- aggregate_rare(ps_rel_abund, taxa_variables["taxa_name_capitalized"], detection = 0, prevalence = taxa_variables["prevalence"])
  ps_rel_abund_taxacore_df <- phyloseq::psmelt(ps_rel_abund_taxacore)
  ps_rel_abund_taxacore_df$AgeGroup13 = factor(ps_rel_abund_taxacore_df$AgeGroup13, levels = AgeGroup13_sorted)
  ps_rel_abund_taxacore_df$EnvironmentGroup5 = factor(ps_rel_abund_taxacore_df$EnvironmentGroup5, levels = EnvironmentGroup5_sorted)
  ps_rel_abund_taxacore_df$Sample <- factor(ps_rel_abund_taxacore_df$Sample, levels = unique(ps_rel_abund_taxacore_df$Sample[order(ps_rel_abund_taxacore_df$AgeGroup13)]))
  return(ps_rel_abund_taxacore_df)
}

ps_rel_abund_taxacore_plot <- function(ps_rel_abund_taxacore_df,taxa_name){
taxa_name_capitalized <- str_to_title(taxa_name)
  if(taxa_name_capitalized == "Phylum"){
    taxa_variables <- c(
      "taxa_name_capitalized" = taxa_name_capitalized,
      "col_set" ="col_phylum",
      "plot_title" = "Relative Abundance of Bacterial Phyla in the Gut Microbiome of Aldabra Giant Tortoises")
  } else if(taxa_name_capitalized == "Family"){
    taxa_variables <- c(
      "taxa_name_capitalized" = taxa_name_capitalized,
      "col_set" ="col_family",
      "plot_title" = "Relative Abundance of Bacterial Families in the Gut Microbiome of Aldabra Giant Tortoises")
  }else if(taxa_name_capitalized == "Genus"){
    taxa_variables <- c(
      "taxa_name_capitalized" = taxa_name_capitalized,
      "col_set" ="col_genus",
      "plot_title" = "Relative Abundance of Bacterial Genera in the Gut Microbiome of Aldabra Giant Tortoises")
  }else if(taxa_name_capitalized == "Species"){
    taxa_variables <- c(
      "taxa_name_capitalized" = taxa_name_capitalized,
      "col_set" ="col_species",
      "plot_title" = "Relative Abundance of Bacterial Species in the Gut Microbiome of Aldabra Giant Tortoises")
  }
  if(!(exists(as.character(taxa_variables["col_set"])))){
    taxa <- unique(ps_rel_abund_taxacore_df[, as.character(taxa_variables["taxa_name_capitalized"])])
    col_set <- data.frame(taxa, distinctColorPalette(length(taxa)))
    col_set <- rbind(col_set, c("Remainder", "grey"))
    rownames(col_set) <- NULL
    colnames(col_set) <- c(as.character(taxa_variables["taxa_name_capitalized"]), "Color")
    class(col_set)
    col_set <- setNames(as.character(col_set[,2]), col_set[,1])
    assign(as.character(taxa_variables["col_set"]), col_set, envir = .GlobalEnv)
  print("you created col set")
  }
  print("you already had col set")
  ggplot(ps_rel_abund_taxacore_df,
       aes(x = Sample, 
           Abundance, 
           fill = fct_reorder(eval(parse(text = taxa_variables["taxa_name_capitalized"])), Abundance, .desc = TRUE))) +
      labs(x = "Age Group", 
           y = "Relative Abundance (%)", 
           title = stringr::str_wrap(taxa_variables["plot_title"], width = 50))+
      facet_grid(cols = vars(EnvironmentGroup5), 
                 scales = "free", 
                 space = "free") +
      theme_bw(base_family = "HiraKakuProN-W3") +
      theme_prism() +
      geom_bar(stat = "identity", 
               position = "fill")+
      scale_fill_manual(as.character(taxa_variables["taxa_name_capitalized"]),
                        values = eval(parse(text = taxa_variables["col_set"]))) +
      scale_y_percent(scale = 100) +
      scale_x_discrete(breaks = ps_rel_abund_taxacore_df$Sample, labels = ps_rel_abund_taxacore_df$AgeGroup13) +
      theme(
        plot.margin = margin(1, 1, 1, 1, "cm"),
        axis.text.x = element_text(angle = 90, color = "black", size = 18),
        axis.text.y = element_text(color = "black", size = 20),
        axis.title = element_text(size = 25),
        legend.position = "bottom",
        legend.title = element_text(size = 20, hjust=0),
        legend.text = element_text(size = 12),
        strip.text.x = element_text(size = 12),
        plot.title = element_text(size = 40, hjust=0.5),
        panel.spacing = unit(1, "lines")) +
      guides(fill = guide_legend(nrow = 9))  
  
}

ps_rel_abund_taxacore_0.01remainder_createdf_plot <- function(ps_rel_abund_taxacore_df, taxa_name){
  taxa_name_capitalized <- str_to_title(taxa_name)
    if(taxa_name_capitalized == "Phylum"){
    taxa_variables <- c(
      "taxa_name_capitalized" = taxa_name_capitalized,
      "col_set" ="col_phylum",
      "plot_title" = "Relative Abundance of Bacterial Phyla (with >1% Abundance) in the Gut Microbiome of Aldabra Giant Tortoises")
  } else if(taxa_name_capitalized == "Family"){
    taxa_variables <- c(
      "taxa_name_capitalized" = taxa_name_capitalized,
      "col_set" ="col_family",
      "plot_title" = "Relative Abundance of Bacterial Families (with >1% Abundance) in the Gut Microbiome of Aldabra Giant Tortoises ")
  }else if(taxa_name_capitalized == "Genus"){
    taxa_variables <- c(
      "taxa_name_capitalized" = taxa_name_capitalized,
      "col_set" ="col_genus",
      "plot_title" = "Relative Abundance of Bacterial Genera (with >1% Abundance) in the Gut Microbiome of Aldabra Giant Tortoises")
  }else if(taxa_name_capitalized == "Species"){
    taxa_variables <- c(
      "taxa_name_capitalized" = taxa_name_capitalized,
      "col_set" ="col_species",
      "plot_title" = "Relative Abundance of Bacterial Species (with >1% Abundance) in the Gut Microbiome of Aldabra Giant Tortoises")
  }
  ps_rel_abund_taxacore_0.1remainder_df <- ps_rel_abund_taxacore_df
  ps_rel_abund_taxacore_0.1remainder_df[,as.character(taxa_variables["taxa_name_capitalized"])] <- as.character(ps_rel_abund_taxacore_0.1remainder_df[,as.character(taxa_variables["taxa_name_capitalized"])])
  medians <- ddply(ps_rel_abund_taxacore_0.1remainder_df, as.quoted(as.character(taxa_variables["taxa_name_capitalized"])), function(x) c(median=median(x$Abundance)))
  remainder <- c(medians[medians$median <= 0.1,as.character(taxa_variables["taxa_name_capitalized"])], "Unknown", "Other")
  ps_rel_abund_taxacore_0.1remainder_df[ps_rel_abund_taxacore_0.1remainder_df[,as.character(taxa_variables["taxa_name_capitalized"])] %in% remainder,as.character(taxa_variables["taxa_name_capitalized"])] <- 'Remainder'
  ps_rel_abund_taxacore_0.1remainder_df$EnvironmentGroup5 = factor(ps_rel_abund_taxacore_0.1remainder_df$EnvironmentGroup5,
                                                                     levels = EnvironmentGroup5_sorted)
  ps_rel_abund_taxacore_0.1remainder_df$Sample <- factor(ps_rel_abund_taxacore_0.1remainder_df$Sample, levels = unique(ps_rel_abund_taxacore_0.1remainder_df$Sample[order(ps_rel_abund_taxacore_0.1remainder_df$AgeGroup13)]))

  if(!(exists(as.character(taxa_variables["col_set"])))){
    taxa <- unique(ps_rel_abund_taxacore_df[, as.character(taxa_variables["taxa_name_capitalized"])])
    col_set <- data.frame(taxa, distinctColorPalette(length(taxa)))
    col_set <- rbind(col_set, c("Remainder", "grey"))
    rownames(col_set) <- NULL
    colnames(col_set) <- c(as.character(taxa_variables["taxa_name_capitalized"]), "Color")
    class(col_set)
    col_set <- setNames(as.character(col_set[,2]), col_set[,1])
    assign(as.character(taxa_variables["col_set"]), col_set, envir = .GlobalEnv)
  print("you created col set")
  }
  print("you already had col set")
  
  ggplot(ps_rel_abund_taxacore_0.1remainder_df,
       aes(x = Sample, 
           Abundance, 
           fill = fct_reorder(eval(parse(text = taxa_variables["taxa_name_capitalized"])), Abundance, .desc = TRUE))) +
      labs(x = "Age Group", 
           y = "Relative Abundance (%)", 
           title = stringr::str_wrap(taxa_variables["plot_title"], width = 60))+
      facet_grid(cols = vars(EnvironmentGroup5), 
                 scales = "free", 
                 space = "free") +
      theme_bw(base_family = "HiraKakuProN-W3") +
      theme_prism() +
      geom_bar(stat = "identity", 
               position = "fill")+
      scale_fill_manual(as.character(taxa_variables["taxa_name_capitalized"]),
                        values = eval(parse(text = taxa_variables["col_set"]))) +
      scale_y_percent(scale = 100) +
      scale_x_discrete(breaks = ps_rel_abund_taxacore_df$Sample, labels = ps_rel_abund_taxacore_df$AgeGroup13) +
        theme(
          plot.margin = margin(1, 1, 1, 1, "cm"),
          axis.text.x = element_text(angle = 90, color = "black", size = 15),
          axis.text.y = element_text(color = "black", size = 15),
          axis.title = element_text(size = 20),
          legend.position = "right",
          legend.title = element_text(size = 15, hjust=0),
          legend.text = element_text(size = 10),
          strip.text.x = element_text(size = 11),
          plot.title = element_text(size = 25, hjust=0.5),
          panel.spacing = unit(1, "lines")) +
        guides(fill = guide_legend(ncol = 1))  
}

beta_ordination_plot <- function(ps_rel_abund, method, distance){
  sample_data(ps_rel_abund)$AgeGroup13 <- factor(sample_data(ps_rel_abund)$AgeGroup13, levels = AgeGroup13_sorted)
  sample_data(ps_rel_abund)$EnvironmentGroup5 <- factor(sample_data(ps_rel_abund)$EnvironmentGroup5, levels = EnvironmentGroup5_sorted)
  method_capitalized <- str_to_upper(method)
  distance_capitalized <- str_to_title(distance)
  if(method_capitalized == "PCOA"){method_capitalized = "PCoA"} else if(method_capitalized == "DPCOA"){method_capitalized = "DPCoA"}
  if(distance_capitalized == "Bray Curtis"){
    distance_variables <- c(
      "distance_capitalized" = distance_capitalized,
      "distance" ="bray")
  } else if(distance_capitalized == "Unweighted Unifrac") {
    distance_variables <- c(
      "distance_capitalized" = distance_capitalized,
      "distance" ="unifrac")    
  } else if(distance_capitalized == "Weighted Unifrac") {
    distance_variables <- c(
      "distance_capitalized" = distance_capitalized,
      "distance" ="wunifrac")    
  }
  ord_pcoa <- ordinate(ps_rel_abund, method = method_capitalized, distance = as.character(distance_variables['distance']))
  plot_ordination(ps_rel_abund, ord_pcoa, 
                  color = "AgeGroup13", 
                  shape = "EnvironmentGroup5", 
                  title = paste(method_capitalized, " plot of Beta diversity (", as.character(distance_variables['distance_capitalized']),")\nof Aldabra Giant Tortoises", sep = "")) +
    theme_prism() +
    geom_point(size = 2.3) +
    scale_shape_manual(values = c(20, 16, 19, 17, 15)) +
    labs(shape = "Envrionment", color = "Age Group") +
    scale_color_manual(values = mycols_agegroup13) +
    theme(
      axis.text.x = element_text(color = "black", size = 25),
      axis.text.y = element_text(color = "black", size = 25),
      axis.title = element_text(size = 30),
      legend.title = element_text(size = 25),
      legend.text = element_text(size = 20),
      strip.text.x = element_text(size = 28),
      plot.title = element_text(size = 30)) +
    stat_ellipse(linetype = 2)
}

venn_taxa_plot <- function(ps_rel_abund, groupby, groupby_order,taxa_name){
  groups <- unique(as.character(microbiome::meta(ps_rel_abund)[,as.character(groupby)]))
  groups <- factor(groups, levels = groupby_order)
  list_core <- c()
  if(taxa_name == "ASVs"){
    for (i in groups){
      ps.sub <- do.call("subset_samples", list(quote(ps_rel_abund), substitute(eval(parse(text = groupby)) == i)))
      core_m <- core_members(ps.sub,
                          detection = 0.001,
                          prevalence = 0.75)
      list_core[[i]] <- core_m 
    }
  } else {
    for (i in groups){
      ps.sub <- do.call("subset_samples", list(quote(ps_rel_abund), substitute(eval(parse(text = groupby)) == i)))
      core_m <- psmelt(aggregate_rare(ps.sub, level = taxa_name, detection = 0, prevalence = 0))
      print(length(core_m[,taxa_name]))
      list_core[[i]] <- core_m[,taxa_name]
    }
  }
  summary(list_core)
  list_core <- list_core[order(levels(groups))]
  ggVennDiagram(list_core, 
                label_alpha = 0, 
                label = "percent", 
                edge_size = 0)+
    ggtitle(paste("Venn Diagram for ", taxa_name, sep="")) +
    theme(
      plot.margin = margin(1, 1, 1, 1, "cm"),
      plot.title = element_text(hjust = 0.5, size = 20, face= "bold"),
      legend.position = "none")+
    scale_fill_gradient(low="gray100",high = "gray50")
}

# Create Phyloseq

### Preprocess Reads

In [ ]:
# Data processing referring to https://benjjneb.github.io/dada2/tutorial.html, https://mibwurrepo.github.io/Microbial-bioinformatics-introductory-course-Material-2018/composition-plots.html

# Primers have already been trimmed using cutadapt
# Forward and reverse fastq filenames have format: SAMPLENAME_R1_001.fastq and SAMPLENAME_R2_001.fastq
fnFs <- sort(list.files(path, pattern = "_R1_001.fastq", full.names = TRUE))
fnRs <- sort(list.files(path, pattern = "_R2_001.fastq", full.names = TRUE))
sample.names <- sapply(strsplit(basename(fnFs), "_"), `[`, 1)

# Place filtered files in filtered/ subdirectory
filtFs <- file.path(path, "filtered", paste0(sample.names, "_F_filt.fastq.gz"))
filtRs <- file.path(path, "filtered", paste0(sample.names, "_R_filt.fastq.gz"))
names(filtFs) <- sample.names
names(filtRs) <- sample.names

# Too few reads passed the filter when using maxEE = c(2,2) so it was relaxed on the reverse reads to maxEE = c(2,5)
out <- filterAndTrim(fnFs, filtFs, fnRs, filtRs,
                     maxN = 0, maxEE = c(2, 5), truncQ = 2, rm.phix = TRUE,
                     compress = TRUE, multithread = TRUE
)
head(out)

# Learn the sequencing error rates
errF <- learnErrors(filtFs, multithread=TRUE)
errR <- learnErrors(filtRs, multithread=TRUE)

dadaFs <- dada(filtFs, err = errF, multithread = TRUE)
dadaRs <- dada(filtRs, err = errR, multithread = TRUE)
mergers <- mergePairs(dadaFs, filtFs, dadaRs, filtRs, verbose = TRUE)

# Constructing ASV table
seqtab <- makeSequenceTable(mergers)
table(nchar(getSequences(seqtab)))

# The expected amplicon size is 260 ~ 460 bp
seqtab <- seqtab[,nchar(colnames(seqtab)) %in% 240:460]

# Chimeric sequences are identified if they can be exactly reconstructed by combining a left-segment and a right-segment from two more abundant “parent” sequences
seqtab.nochim <- removeBimeraDenovo(seqtab, method = "consensus", multithread = TRUE, verbose = TRUE)
sum(seqtab.nochim)/sum(seqtab)
# Tracking reads throught the pipeline
getN <- function(x) sum(getUniques(x))
track <- cbind(out, sapply(dadaFs, getN), sapply(dadaRs, getN), sapply(mergers, getN), rowSums(seqtab.nochim))
colnames(track) <- c("input", "filtered", "denoisedF", "denoisedR", "merged", "nonchim")
rownames(track) <- sample.names
head(track)
sum(track[,"input"])
sum(track[,"nonchim"])
```

```{r Assign Taxonomy}
taxa_GTDB <- assignTaxonomy(seqtab.nochim, "GTDB_ssu_all_r214_cleaned_ready.fa.gz", multithread = TRUE)
taxa.print <- taxa_GTDB
rownames(taxa.print) <- NULL
head(taxa_GTDB)

### Construct Phylogenetic Tree

In [ ]:
sequences <- getSequences(seqtab.nochim)
names(sequences) <- sequences
alignment <- AlignSeqs(DNAStringSet(sequences), anchor = NA)
phang.align <- phyDat(as(alignment, "matrix"), type = "DNA")
dm <- dist.ml(phang.align)
treeNJ <- NJ(dm)
fit <- pml(treeNJ, data = phang.align)
fitGTR <- update(fit, k = 4, inv = 0.2)

save.image(file = "20231021-AldabraGiantTortoises-dada2.RData")

fitGTR <- optim.pml(fitGTR,
                    model = "GTR", optInv = TRUE, optGamma = TRUE,
                    rearrangement = "stochastic", control = pml.control(trace = 0)
)

### Add Metadata

In [ ]:
samdf <- read.table("20230113 sample_metadata.tsv", sep = "\t", header = T, row.names = 1, comment = "")
samdf$AgeGroup <- gsub(" (", "\n(", samdf$AgeGroup, fixed = TRUE)
head(samdf)
# Preparing phyloseq
ps <- phyloseq(
  otu_table(seqtab.nochim, taxa_are_rows = FALSE),
  sample_data(samdf),
  tax_table(taxa_GTDB), phy_tree(fitGTR$tree)
)
# Storing the DNA sequences of our ASVs in the refseq slot of the phyloseq object,then renaming our taxa to a short string. That way, the short new taxa names will appear in tables and plots, and we can still recover the DNA sequences corresponding to each ASV as needed with refseq(ps)
dna <- Biostrings::DNAStringSet(taxa_names(ps))
names(dna) <- taxa_names(ps)
ps <- merge_phyloseq(ps, dna)
taxa_names(ps) <- paste0("ASV", seq(ntaxa(ps)))

sample_data(ps)$Sex <- factor(sample_data(ps)$Sex, levels = c("Not determined", "F", "M"))
sample_data(ps)$EnvironmentGroup5 <- factor(sample_data(ps)$EnvironmentGroup5, levels = EnvironmentGroup5_sorted)

# Adding the ASV ids from ASV table into the taxa table at the end
taxa_names(ps) <- paste0("ASV_", rownames(tax_table(ps)))
taxic <- as.data.frame(ps@tax_table) # this will help in setting large color options
taxic$ASV <- rownames(taxic)
colnames(taxic)
taxmat <- as.matrix(taxic)
new.tax <- tax_table(taxmat)
tax_table(ps) <- new.tax
tax_table(ps)[tax_table(ps)[, "Family"] == "", "Family"] <- "Unclassified family"

### Subset Phyloseq

In [ ]:
ps_bg_cr <- subset_samples(ps, EnvironmentGroup5 == "Botanical\nGarden" | EnvironmentGroup5 == "Curieuse")
ps_wild <- subset_samples(ps, EnvironmentGroup5 == "Curieuse" )
ps_controlled <- subset_samples(ps, EnvironmentGroup5 != "Curieuse")
ps_bg <- subset_samples(ps, EnvironmentGroup5 == "Botanical\nGarden")

### Phyloseq Statistics

In [ ]:
summarize_phyloseq(ps)
print(ps)
length(unique(tax_table(ps)[,'Phylum']))
length(unique(tax_table(ps)[,'Class']))
length(unique(tax_table(ps)[,'Family']))
length(unique(tax_table(ps)[,'Genus']))
length(unique(tax_table(ps)[,'ASV']))

# Save R Image

In [ ]:
save.image(file = R_image)

# Alpha Diversity

In [ ]:
# Alpha diversity - whole dataset
ps_df_rich <- alpha_diversity_calculation(ps)
ps_df_rich$EnvironmentGroup5 <- factor(ps_df_rich$EnvironmentGroup5, levels = EnvironmentGroup5_sorted)

alpha_diversity_plot(ps_df_rich, "EnvironmentGroup5", "Shannon", "Curieuse", "AgeGroup13", mycols_agegroup13, "\nEnvironment", "Shannon", "Alpha Diversity of Aldabra Giant Tortoises from Different Envrionemnts", "Age Group")

alpha_diversity_plot(ps_df_rich, "EnvironmentGroup5", "InvSimpson", "Curieuse", "AgeGroup13", mycols_agegroup13, "\nEnvironment", "Inverted Simpson", "Alpha Diversity of Aldabra Giant Tortoises from Different Envrionemnts", "Age Group")

alpha_diversity_plot(ps_df_rich, "EnvironmentGroup5", "Shannon", "Curieuse", "AgeGroup13", mycols_agegroup13, "\n環境", "Shannon", "様々な環境のアルダブラゾウガメの\n腸内細菌のアルファ多様性（Shannon)", "年齢層")


# Alpha diversity - wild
ps_df_rich_wild <- alpha_diversity_calculation(ps_wild)

alpha_diversity_plot(ps_df_rich_wild, "AgeGroup13", "Shannon", "100",  "EnvironmentGroup5", mycols_environment5[3], "\nAge Group", "Shannon", "Alpha Diversity (Shannon) of Aldabra Giant Tortoises from Wild Envrionemnt", "Environment")

alpha_diversity_plot(ps_df_rich_wild, "AgeGroup13", "Shannon", "100",  "EnvironmentGroup5", mycols_environment5[3], "\n年齢層", "Shannon", "様々な年齢層の野生アルダブラゾウガメの\n腸内細菌のアルファ多様性（Shannon)", "環境")


# Alpha diversity - controlled
ps_df_rich_controlled <- alpha_diversity_calculation(ps_controlled)
ps_df_rich_controlled$EnvironmentGroup5 <- factor(ps_df_rich_controlled$EnvironmentGroup5, levels = EnvironmentGroup5_sorted[c(1,2,4,5)])

alpha_diversity_plot(ps_df_rich_controlled, "AgeGroup13", "Shannon", "100",  "EnvironmentGroup5", mycols_environment5[c(1,2,4,5)], "\nAge Group", "Shannon", "Alpha Diversity (Shannon) of Aldabra Giant Tortoises from Controlled Envrionemnts", "Environment")

alpha_diversity_plot(ps_df_rich_controlled, "AgeGroup13", "Shannon", "100",  "EnvironmentGroup5", mycols_environment5[c(1,2,4,5)], "\n年齢層", "Shannon", "様々な年齢層の飼育アルダブラゾウガメの\n腸内細菌のアルファ多様性（Shannon)", "環境")


# Alpha diversity - Botanical Garden
ps_df_rich_bg <- alpha_diversity_calculation(ps_bg)

alpha_diversity_plot(ps_df_rich_bg, "AgeGroup13", "Shannon", "100",  "EnvironmentGroup5", mycols_environment5[4], "\nAge Group", "Shannon", "Alpha Diversity (Shannon) of Aldabra Giant Tortoises from Botanical Garden", "Environment")


# Alpha diversity - Botanical Gardn + Curieuse
ps_df_rich_bg_cr <- alpha_diversity_calculation(ps_bg_cr)
ps_df_rich_bg_cr$EnvironmentGroup5 <- factor(ps_df_rich_bg_cr$EnvironmentGroup5, levels = EnvironmentGroup5_sorted[c(3,4)])

alpha_diversity_plot(ps_df_rich_bg_cr, "AgeGroup13", "Shannon", "100",  "EnvironmentGroup5", mycols_environment5[c(3,4)], "\nAge Group", "Shannon", "Alpha Diversity (Shannon) of Aldabra Giant Tortoises from Controlled Envrionemnts", "Environment")

# Beta Diversity

In [ ]:
beta_ordination_plot(ps_rel_abund, "pcoa", "bray curtis")

beta_ordination_plot(ps_rel_abund, "pcoa", "unweighted unifrac")

beta_ordination_plot(ps_rel_abund, "pcoa", "weighted unifrac")

# Relative Abundance

In [ ]:
ps_rel_abund <- transform_sample_counts(ps, function(otu) {otu * 100 / sum(otu, na.rm = T)})
ps_rel_abund_wild <- transform_sample_counts(ps_wild, function(otu) {otu * 100 / sum(otu, na.rm = T)})
ps_rel_abund_controlled <- transform_sample_counts(ps_controlled, function(otu) {otu * 100 / sum(otu, na.rm = T)}) 

# Phylum
ps_rel_abund_phylumcore_df <- ps_rel_abund_taxacore_createdf(ps_rel_abund, "phylum")
ps_rel_abund_taxacore_plot(ps_rel_abund_phylumcore_df, "phylum")
ps_rel_abund_taxacore_0.01remainder_createdf_plot(ps_rel_abund_phylumcore_df,"phylum")

# Family
ps_rel_abund_familycore_df <- ps_rel_abund_taxacore_createdf(ps_rel_abund, "family")
ps_rel_abund_taxacore_0.01remainder_createdf_plot(ps_rel_abund_familycore_df,"family")

# Genera
ps_rel_abund_genuscore_df <- ps_rel_abund_taxacore_createdf(ps_rel_abund, "genus")
ps_rel_abund_taxacore_0.01remainder_createdf_plot(ps_rel_abund_genuscore_df,"genus")

# Venn Diagram

In [ ]:
# Venn diagram for the 4 main age groups
venn_taxa_plot(ps_rel_abund, "AgeGroup", AgeGroup_vennsorted, "ASVs")
venn_taxa_plot(ps_rel_abund, "AgeGroup", AgeGroup_vennsorted, "Genus")
venn_taxa_plot(ps_rel_abund, "AgeGroup", AgeGroup_vennsorted, "Species")
venn_taxa_plot(ps_rel_abund, "AgeGroup", AgeGroup_vennsorted, "Phylum")
venn_taxa_plot(ps_rel_abund, "AgeGroup", AgeGroup_vennsorted, "Family")

venn_taxa_plot(ps_rel_abund_wild, "AgeGroup", AgeGroup_vennsorted_wild, "ASVs")
venn_taxa_plot(ps_rel_abund_wild, "AgeGroup", AgeGroup_vennsorted_wild, "Genus")
venn_taxa_plot(ps_rel_abund_wild, "AgeGroup", AgeGroup_vennsorted_wild, "Species")

venn_taxa_plot(ps_rel_abund_controlled, "AgeGroup", AgeGroup_vennsorted_controlled, "Genus")
venn_taxa_plot(ps_rel_abund_controlled, "AgeGroup", AgeGroup_vennsorted_controlled, "Species")
venn_taxa_plot(ps_rel_abund_controlled, "AgeGroup", AgeGroup_vennsorted_controlled, "ASVs")

# Venn diagram for the 4 main environment groups
venn_taxa_plot(ps_rel_abund, "EnvironmentGroup5",EnvironmentGroup5_vennsorted, "ASVs")
venn_taxa_plot(ps_rel_abund, "EnvironmentGroup5",EnvironmentGroup5_vennsorted, "Genus")
venn_taxa_plot(ps_rel_abund, "EnvironmentGroup5",EnvironmentGroup5_vennsorted,"Species")
venn_taxa_plot(ps_rel_abund, "EnvironmentGroup5",EnvironmentGroup5_vennsorted,"Phylum")
venn_taxa_plot(ps_rel_abund, "EnvironmentGroup5",EnvironmentGroup5_vennsorted,"Family")